# News Articles Categorisation


The steps to build the model are

- Data Exploration
- Text Processing (Cleaning)
- Feature Extraction
- Modelling
- Use the Model

# Possible Reference


- https://www.analyticsvidhya.com/blog/2021/08/malawi-news-classification-an-nlp-project/

# Importing Libraries

In [1]:
import nltk

# nltk.download()

In [4]:
# Import libraries

import pandas as pd
import re 

import nltk 
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost 
from sklearn.metrics  import classification_report
from sklearn import metrics
import time



# Combining multiple excel files into one dataframe


In [34]:
import os
import pandas as pd
cwd = os.path.abspath('CSV data') 
files = os.listdir(cwd) 

In [35]:
cwd

'C:\\Users\\Darren\\Documents\\GitHub\\FSX\\News Categorisation\\CSV data'

In [36]:
files

['avalanche_straitstimes.csv',
 'dengue_straitstimes.csv',
 'pandemic_straitstimes.csv',
 'volcanic eruption_straitstimes.csv']

In [12]:
df = pd.DataFrame()
for file in files:
     if file.endswith('.csv'):
         df = df.append(pd.read_csv(file), ignore_index=True) 
df.head()

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name
0,0,02/2022,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities
1,1,02/2022,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities
2,2,02/2022,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities
3,3,02/2022,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities
4,4,02/2021,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities


In [14]:
df.shape

(1316, 8)

In [16]:
df['class_name'].value_counts()

natural calamities     776
Human health crises    540
Name: class_name, dtype: int64

# Data Exploration

In [18]:
category = list(df['class_name'].unique())
category

['natural calamities', 'Human health crises']

# Text Preprocessing

In [21]:
def preprocess(text):
    
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the article
    Return:
      lem(list of str): a list of the root form of the article words
    """
        
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", str(text).lower())
    
    # Tokenize text
    token = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in token if t not in stop]
    
    # Lemmatization
    lem = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lem

In [22]:
df["Preprocessed_Text"] = df['content summary'].apply(lambda x: preprocess(x))
df.head(10)

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,Preprocessed_Text
0,0,02/2022,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,"[mumbai, reuters, himalayan, avalanche, killed..."
1,1,02/2022,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,"[zurich, reuters, one, person, killed, four, o..."
2,2,02/2022,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,"[vienna, reuters, avalanche, area, austria, bo..."
3,3,02/2022,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,"[new, delhi, reuters, himalayan, avalanche, tr..."
4,4,02/2021,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,"[salt, lake, city, nytimes, four, back, countr..."
5,5,04/2021,NEW DELHI (XINHUA),8 killed after glacial burst triggers avalanch...,https://www.straitstimes.com/asia/south-asia/8...,"According to officials, 384 others, mostly bel...","avalanche, officials, bro, glacial, uttarakhan...",natural calamities,"[according, official, others, mostly, belongin..."
6,6,01/2021,VIENNA (REUTERS),Dogs' barking prompts owners' rescue from Swis...,https://www.straitstimes.com/world/europe/dogs...,VIENNA (REUTERS) - Two people caught in an ava...,"dogs, avalanche, buried, snowshoers, group, wi...",natural calamities,"[vienna, reuters, two, people, caught, avalanc..."
7,7,01/2021,FRANCE (AFP),'Miracle' escape for man trapped in Alps avala...,https://www.straitstimes.com/world/europe/mira...,FRANCE (AFP) - A man out walking with his fami...,"minutes, avalanche, local, escape, snow, rescu...",natural calamities,"[france, afp, man, walking, family, french, al..."
8,8,01/2020,"MUZAFFARABAD, PAKISTAN (REUTERS)",Girl buried under Pakistan avalanche for 18 ho...,https://www.straitstimes.com/asia/south-asia/g...,"MUZAFFARABAD, PAKISTAN (REUTERS) - A 12-year-o...","buried, avalanche, waited, pakistan, village, ...",natural calamities,"[muzaffarabad, pakistan, reuters, year, old, g..."
9,9,12/2019,VIENNA (REUTERS),"Rescuers comb Austrian, Swiss avalanches in ca...",https://www.straitstimes.com/world/europe/resc...,VIENNA (REUTERS) - Rescuers hunted for possibl...,"avalanche, large, snow, warning, swiss, victim...",natural calamities,"[vienna, reuters, rescuer, hunted, possible, v..."


# Text Exploration

In [23]:
def find_common_words(df, category):
        
    """
    Function: find the most frequent words in the category and return the them
    Args:
      df(dataframe): the dataframe of articles
      category(str): the category name
    Return:
      the most frequant words in the category
    """
        
    # Create dataframes for the category
    cat_df = df[df["class_name"]==category]
    
    # Initialize words list for the category
    words = [word for tokens in cat_df["Preprocessed_Text"] for word in tokens]
    
    # Count words frequency
    words_counter = Counter(words)
 
    return words_counter.most_common(10)

In [25]:
print("Most common words in each category")
for c in category:
    print(c, " News")
    print(find_common_words(df, c))
    print()

Most common words in each category
natural calamities  News
[('said', 904), ('volcano', 554), ('eruption', 423), ('people', 397), ('avalanche', 308), ('mr', 300), ('year', 269), ('new', 265), ('island', 248), ('ash', 231)]

Human health crises  News
[('dengue', 955), ('year', 578), ('case', 515), ('said', 456), ('singapore', 381), ('mosquito', 353), ('pandemic', 315), ('number', 261), ('last', 259), ('covid', 238)]



In [26]:
df['Preprocessed_Text2'] = df['Preprocessed_Text'].apply(' '.join)
df.head()

,Unnamed: 0,date,location,news title,news source(url),content summary,keywords,class_name,Preprocessed_Text,Preprocessed_Text2
0,0,02/2022,MUMBAI (REUTERS),Himalayan avalanche kills seven Indian soldier...,https://www.straitstimes.com/asia/himalayan-av...,MUMBAI (REUTERS) - A Himalayan avalanche kille...,"kills, avalanche, defence, kameng, china, sold...",natural calamities,"[mumbai, reuters, himalayan, avalanche, killed...",mumbai reuters himalayan avalanche killed seve...
1,1,02/2022,ZURICH (REUTERS),Eight killed in two days after third deadly av...,https://www.straitstimes.com/world/europe/eigh...,ZURICH (REUTERS) - One person was killed and f...,"person, avalanche, tyrol, killed, days, skiers...",natural calamities,"[zurich, reuters, one, person, killed, four, o...",zurich reuters one person killed four others i...
2,2,02/2022,VIENNA (REUTERS),Avalanche in Austria near Swiss border kills five,https://www.straitstimes.com/world/europe/aval...,VIENNA (REUTERS) - An avalanche in an area of ...,"kills, avalanche, person, services, supervisor...",natural calamities,"[vienna, reuters, avalanche, area, austria, bo...",vienna reuters avalanche area austria borderin...
3,3,02/2022,NEW DELHI (REUTERS),Himalayan avalanche traps Indian Army patrol t...,https://www.straitstimes.com/asia/south-asia/h...,NEW DELHI (REUTERS) - A Himalayan avalanche tr...,"arunachal, avalanche, army, defence, team, chi...",natural calamities,"[new, delhi, reuters, himalayan, avalanche, tr...",new delhi reuters himalayan avalanche trapped ...
4,4,02/2021,SALT LAKE CITY (NYTIMES),"4 skiers killed in avalanche in Utah, official...",https://www.straitstimes.com/world/united-stat...,SALT LAKE CITY (NYTIMES) - Four back-country s...,"avalanche, states, killed, skiers, saidthe, sa...",natural calamities,"[salt, lake, city, nytimes, four, back, countr...",salt lake city nytimes four back country skier...


In [27]:
X = df['Preprocessed_Text2']
y = df['class_name']

# Feature Extraction

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [29]:
tf_vec = TfidfVectorizer()
train_features = tf_vec.fit(X_train)
train_features = tf_vec.transform(X_train)
test_features = tf_vec.transform(X_test)

# Modelling

In [30]:
def fit_eval_model(model, train_features, y_train, test_features, y_test):
    
    """
    Function: train and evaluate a machine learning classifier.
    Args:
      model: machine learning classifier
      train_features: train data extracted features
      y_train: train data lables
      test_features: train data extracted features
      y_test: train data lables
    Return:
      results(dictionary): a dictionary of the model training time and classification report
    """
    results ={}
    
    # Start time
    start = time.time()
    # Train the model
    model.fit(train_features, y_train)
    # End time
    end = time.time()
    # Calculate the training time
    results['train_time'] = end - start
    
    # Test the model
    train_predicted = model.predict(train_features)
    test_predicted = model.predict(test_features)
    
     # Classification report
    results['classification_report'] = classification_report(y_test, test_predicted)
        
    return results

In [31]:
sv = svm.SVC()
ab = AdaBoostClassifier(random_state = 1)
gb = GradientBoostingClassifier(random_state = 1)
xgb = xgboost.XGBClassifier(random_state = 1)
tree = DecisionTreeClassifier()
nb = MultinomialNB()


# Fit and evaluate models
results = {}
for cls in [sv, ab, gb, xgb, tree, nb]:
    cls_name = cls.__class__.__name__
    results[cls_name] = {}
    results[cls_name] = fit_eval_model(cls, train_features, y_train, test_features, y_test)

C:\Users\Darren\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:44:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [32]:
for res in results:
    print (res)
    print()
    for i in results[res]:
        print (i, ':')
        print(results[res][i])
        print()
    print ('-----')
    print()

SVC

train_time :
0.6494221687316895

classification_report :
                     precision    recall  f1-score   support

Human health crises       0.97      0.87      0.92       110
 natural calamities       0.92      0.98      0.95       154

           accuracy                           0.94       264
          macro avg       0.94      0.93      0.93       264
       weighted avg       0.94      0.94      0.94       264


-----

AdaBoostClassifier

train_time :
0.4564037322998047

classification_report :
                     precision    recall  f1-score   support

Human health crises       0.95      0.86      0.90       110
 natural calamities       0.91      0.97      0.94       154

           accuracy                           0.92       264
          macro avg       0.93      0.92      0.92       264
       weighted avg       0.93      0.92      0.92       264


-----

GradientBoostingClassifier

train_time :
1.673257827758789

classification_report :
                     pr